# Задача

**Требуется:** предложить модель, сегментирующую человека на фотографии (без использования предобученных и готовых моделей).  
  
**Вход:** фотография 320x240x3.  
**Выход:** маска человека 320x240.  
**Метрика:** [Dice coefficient](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient).  
  

Данные представляют из себя набор фотографий человека и маски, определяющей положение человека на фотографии.  
Доступные данные разделены на несколько папок:  
- `train` содержит фотографии 320x240x3;
- `train_mask` содержит маски для фотографий из `train` 320x240;
- `valid` содержит фотографии 320x240x3;
- `valid_mask` содержит маски для фотографий из `valid` 320x240;
- `test` содержит фотографии 320x240x3.  
  
Для лучшей модели требуется создать 2 файла, которые необходимы для валидации решения:
- сохраненные маски для картинок из valid в формате pred_valid_template.csv (в архиве с `data`);
- html страницу с предсказанием модели для всех картинок из test и папку с используемыми картинками в этой html странице для её просмотра.

# Решение

Так как в задании прописаны требования по созданию сети с нуля, то нет возможности использовать любые предобученные модели или архитектуры из коробки, (классификационные предобученные сети сильно бы помогли). 

Для функции потерь dice loss подходит слабо, так как она слишком сильно меняется в зависимости от пересечения и не дает хорошей дополнительной информации для обучения. Изначально была выбрана попиксельный L1 loss c дальнейшим добавлением коэфициента зоны перехода границы маски, чтобы сеть больше внимания уделяла области между человеком и фоном. Коэфициент зоны меняется в зависимости от степени обучения.

За baseline была взята собранная с нуля архитектура Unet так как у нас бинарная классификация, которая достаточно похожа на медицинские данные предназначенные для Unet.

Слишком глубокий Unet плохо обучается и часто вылетает из-за недостатка видеопамяти. Выбрана глубина 8, 16, 32, 64, 128 слоем на каждом уровне соответственно

В качестве оптимизатора был взят Adam со скоростью обучения 0.0001. При более высокой скорости обучения сеть не успевает найти нужные признаки и сваливается в наиболее вероятные положения масок во всем датасете, то есть начинает предсказывать прямоугольник по центру для любой фото. 

Добавление dropout 2D на центральный боттлнек не дало улучшения.
То же самое относится к BatchNorm, который приводит к нестабильности обучения (хотя в теории должно быть наоборот)

Из аугментации были выбранны только базовые измения цветов и небольшие трансформации из пакета torchvision, так как любая, чуть более сильная аугментация, приводит сеть к осциляции между полной маской и полностью пустой маской.

Гипотеза о начальном обучении сети на фото с вырезанным человеком и дальнейшим плавным проявлением фона в зависимости от метрики не привела к улучшению, так как сеть выучивала только границу между фоном и маской и отказывалась искать другие признаки.

Дальше в архитектуру были добавлены 2 residual connections между входным и  классифицирующим выходными слоем.
Финальный слой был расширен. Так же был  добавлен еще один классификатор (тоже сверка 1х1, для повышения вариативности при увеличении количества слов и преодоление XOR-подобных проблем). Это добавило стабильности обучения.

Использованние маскирующих сверток  на входе сети позволяет прибавить до 1% dice, но приводит к сильной нестабильности  и затыкам обучения. В финальной модели они не используются. 

Добавлены Squeeze-and-Excitation блоки, так как в бинарной сегментации хорошо себя показывают использование предобученных архитектур на основе SE слоев (например SE-ResNeXt-50). Это улучшило dice на 2%.

Для предобработки входных фото используются кастомные сверточные слои с фиксированными весами распределения Гаусса имеющие различный сдвиг по амлитуде. Это позволяет выделять грани переходов, обратные грани, и общий цвет зоны для каждого канала индивидуально. Применяя свертки различного размера можно получить подобие спектральных каналов. Были выбранные 3 размера сверток размерами 5, 9 и 21 пиксель, что все вместе дает на вход сети 27 канальное изображение. 


In [ ]:
import platform
if platform.system() == 'Linux':
    path0 = '/content/drive/My Drive/Colab Notebooks/MFTI/test/mipt_test_git'
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.insert(1, path0)
else:
    path0 = '/MFTI/test/mipt_test_git'

In [ ]:
import os
os.chdir("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob   

from lib import *

import tqdm
import pickle
import time
import pandas as pd

import torch as tr
import torch.fft as trf
import torch.distributions as trd
import torchvision.transforms as tvt
t = tr.tensor

%matplotlib inline

In [ ]:
res = []
to_tensor_ = tvt.ToTensor()

for i, data_type in enumerate(['train', 'valid']):
    images = os.listdir(path0 + f'/data/{data_type}')
    r = []
    for im in tqdm.auto.tqdm(images):
        i = (to_tensor_(Image.open(f'{path0}/data/{data_type}/{im}')), 
            to_tensor_(Image.open(f"{path0}/data/{data_type}_mask/{im.split('.')[0]}.png")))
        r.append(i)
    res.append(r)

In [ ]:
class SE(tr.nn.Module):
    """Squeeze-and-Excitation block
    https://arxiv.org/pdf/1709.01507.pdf
    
    """

    def __init__(self, ch, r=16):
        super().__init__()
        self.avg_pool = tr.nn.AdaptiveAvgPool2d(1)
        self.fc = tr.nn.Sequential(
            tr.nn.Linear(ch, ch // r, bias=False),
            tr.nn.ReLU(inplace=True),
            tr.nn.Linear(ch // r, ch, bias=False),
            tr.nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [ ]:
class Unet(tr.nn.Module):
    
    def __init__(self, n=[1, 32, 64, 128, 256, 512], padd = 'same', err = False):
        super().__init__()
        self.err = err
        self.n = n
        
        # self.mask11 = tr.nn.Conv2d(9, n[1], 3, padding = padd)
        # self.mask12 = tr.nn.Conv2d(n[1], n[1], 3, padding = padd)
        # self.mask13 = tr.nn.Conv2d(n[1], n[1], 3, padding = padd)

        self.conv11 = tr.nn.Conv2d(27, n[1], 3, padding = padd)
        self.conv12 = tr.nn.Conv2d(n[1], n[1], 3, padding = padd)
        self.se13 = SE(n[1])

        # self.mask20 = tr.nn.Conv2d(n[1], n[2], 3, padding = padd)
        self.conv21 = tr.nn.Conv2d(n[1], n[2], 3, padding = padd)
        self.conv22 = tr.nn.Conv2d(n[2], n[2], 3, padding = padd)
        self.se23 = SE(n[2])

        self.conv31 = tr.nn.Conv2d(n[2], n[3], 3, padding = padd)
        self.conv32 = tr.nn.Conv2d(n[3], n[3], 3, padding = padd)
        self.se33 = SE(n[3])

        self.conv41 = tr.nn.Conv2d(n[3], n[4], 3, padding = padd)
        self.conv42 = tr.nn.Conv2d(n[4], n[4], 3, padding = padd)
        self.se43 = SE(n[4])

        self.conv51 = tr.nn.Conv2d(n[4], n[5], 3, padding = padd)
        self.conv52 = tr.nn.Conv2d(n[5], n[5], 3, padding = padd)
        self.se53 = SE(n[5])

        self.deconv60 = tr.nn.ConvTranspose2d(n[5], n[4], 3, padding=padd, output_padding=padd,   stride=2)
        self.conv61 = tr.nn.Conv2d(n[5], n[4], 3, padding = padd)
        self.conv62 = tr.nn.Conv2d(n[4], n[4], 3, padding = padd)
        self.se63 = SE(n[4])

        self.deconv70 = tr.nn.ConvTranspose2d(n[4], n[3], 3, padding=padd, output_padding=padd, stride=2)
        self.conv71 = tr.nn.Conv2d(n[4], n[3], 3, padding = padd)
        self.conv72 = tr.nn.Conv2d(n[3], n[3], 3, padding = padd)
        self.se73 = SE(n[3])

        self.deconv80 = tr.nn.ConvTranspose2d(n[3], n[2], 3, padding=padd, output_padding=padd, stride=2)
        self.conv81 = tr.nn.Conv2d(n[3], n[2], 3, padding = padd)
        self.conv82 = tr.nn.Conv2d(n[2], n[2], 3, padding = padd)
        self.se83 = SE(n[2])

        self.deconv90 = tr.nn.ConvTranspose2d(n[2], n[1], 3, padding=padd, output_padding=padd, stride=2)
        self.conv91 = tr.nn.Conv2d(n[2]+27, n[1], 3, padding = padd)
        self.conv92 = tr.nn.Conv2d(n[1], n[1], 3, padding = padd)
        self.se93 = SE(n[1])

        self.conv100 = tr.nn.Conv2d(n[1]+27, n[-1], 1)
        self.se103 = SE(n[-1])
        if self.err:
            self.conv110 = tr.nn.Conv2d(n[-1], 2, 1)
        else:
            self.conv110 = tr.nn.Conv2d(n[-1], 1, 1)


        self.maxpool = tr.nn.MaxPool2d(2)
        self.bn = tr.nn.BatchNorm2d(n[5])
        self.m0 = 0

    def forward(self, input, dropout_koef):

        dropout  = tr.nn.Dropout2d(dropout_koef)

        r = tr.nn.ReLU()
        sig = tr.nn.Sigmoid()

        x = input

        # m = r(self.mask11(x))
        # m = (self.mask12(m))
        # m = r(self.mask13(m))
        if str(type(self.m0)) in "<class 'torch.Tensor'>" :
          x *= self.m0.mean(dim=0).unsqueeze(0)

        x = r(self.conv11(x))
        # x = r(m * x)
        x = r(self.conv12(x))
        x = self.se13(x)
        x1 = x
        x = self.maxpool(x1)

        # m = r(self.mask20(x))
        x = r(self.conv21(x))
        # x *= m
        x = r(self.conv22(x))
        x = self.se23(x)
        x2 = x
        x = self.maxpool(x2)
        
        x = r(self.conv31(x))
        x = r(self.conv32(x))
        x = self.se33(x)
        x3 = x
        x = self.maxpool(x3)
       
        x = r(self.conv41(x))
        x = r(self.conv42(x))
        x = self.se43(x)
        x4 = x
        x = self.maxpool(x4)
       
        x = r(self.conv51(x))
        x = r(self.conv52(x))
        x = self.se53(x)
        x5 = x
        
        x = tr.cat([self.deconv60(x5), x4], dim=1)
        x = r(self.conv61(x))
        x6 = r(self.conv62(x))
        x = self.se63(x)

        x = tr.cat([self.deconv70(x6), x3], dim=1)
        x = r(self.conv71(x))
        x7 = r(self.conv72(x))
        x = self.se73(x)

        x = tr.cat([self.deconv80(x7), x2], dim=1)
        x = r(self.conv81(x))
        x8 = r(self.conv82(x))
        x = self.se83(x)

        x = tr.cat([self.deconv90(x8), x1, input], dim=1)
        x = r(self.conv91(x))
        x9 = r(self.conv92(x))
        x = self.se93(x)

        x = x9
        x = tr.cat([x, input], dim=1)

        x = sig(self.conv100(x))

        # err = x[:,0:1]

        x = self.se103(x)

        err = x[:,0:1]
        m = x[:,1:2]
        x1 = x[:,1:2]
        x = x * m

        size = int(x.shape[1] / 2)
        m = x[:,:size]
        x1 = x[:,size:]
        x = tr.cat([x1, x1 * m], dim = 1)

        x = sig(self.conv110(x))

        pred = x[:, 0].unsqueeze(1) 
        # err = x[:, 1].unsqueeze(1)

        if self.err:
            return pred, err
        else:
            return pred

In [ ]:
dev = tr.device('cuda:0' if tr.cuda.is_available() else 'cpu')
print(f"work on {(tr.cuda.get_device_name() if dev.type == 'cuda' else 'cpu')}")

In [ ]:
def dice(y, pred):
    """Dice loss
    
    Args:
        y (tensor): true mask [batch_size, 1, H, W]
        pred (tensor): pred mask [batch_size, 1, H, W]
    Returns:
        dice_loss (tensor): [1]
    """

    true = (y >= 0.5).to(dev)
    pred = pred >= 0.5
    intersection = (true * pred).sum()
    im_sum = true.sum() + pred.sum()
    return 2.0 * intersection / (im_sum + 1e-10)

In [ ]:
def create_conv_kernels(types_params, X_shape):
    """Make tensor of conv kernels as 2D normal distributions 
    with 5 parameters (E_x, E_y, D_x, D_y, bias) 
    from types_params tensor.

    Args:
        types_params (tensor): gaussian params [num_kernels, 5].
        X_shape (tensor.shape): data shapes for kernel size.

    Returns:
        Tensor of Gaussian kernels with shapes
        [num_kernels, data.shape[-1], data.shape[-2]] 
    """
        
    im_center = (t(X_shape[-2:]) / 2).int()
    kernels = []
    for params in types_params:
        E_xy = params[:2] + im_center
        D_xy = tr.diag(params[2:4])
        grid_xy = tr.cartesian_prod(
            tr.arange(X_shape[-2]), 
            tr.arange(X_shape[-1])
            ).reshape(*X_shape[-2:], 2)
        bias = params[4]
        # print(D_xy)
        kernel = trd.MultivariateNormal(
            E_xy, D_xy
            ).log_prob(grid_xy).exp()#.to(dev)
        kernel -= kernel.mean() * bias
        kernel /= kernel.max()
        kernels.append(kernel.unsqueeze(0))
    return tr.cat(kernels, dim=0)


In [ ]:
def augments(x, y, aug_koef=0.5):
    """Make augmentations:
    RandomPerspective, RandomAffine, 
    RandomInvert, ColorJitter and color channels permutations

    Args:
        X (tensor): images [batch_size, channels, H, W]
        y (tensor): true mask [batch_size, 1, H, W]
        aug_koef (int): aug probability

    Returns:
        X (tensor): augmented images [batch_size, channels, H, W]
        y (tensor): augmented true mask [batch_size, 1, H, W]
    """
    aug_koef = aug_koef % 1
    y1 = y  
    # xy = tr.cat([x, y], dim=1).to(dev)
    # xy = tvt.RandomPerspective(distortion_scale=0.3, p=aug_koef,  fill=0)(xy)
    # xy = tvt.RandomAffine(30 * aug_koef, translate = (aug_koef, aug_koef))(xy)
    # y1 = xy[:, -1].unsqueeze(1)
    # x = xy[:, :-1]
    
    # x = tvt.transforms.RandomInvert(p=aug_koef)(x)
    # x = x[:, tr.randperm(3),:,:]
    # diap = ((1-aug_koef), 1*(1-aug_koef))
  
    x = tvt.ColorJitter(brightness=(0.5, 2), contrast=(0.5, 2) , 
                        saturation=(0.5, 2), hue=(-0.2, 0.2))(x)
    return x, y1
    
def blend_xy(X, y, koef):
    """Add some y information to X, for quick start 

    Args:
        X (tensor): images [batch_size, channels, H, W]
        y (tensor): true mask [batch_size, 1, H, W]
        koef (int): mix koef (1 - clear X, 0 - clear y)

    Returns:
        X (tensor): images [batch_size, channels, H, W]
    """

    return X * koef +  y * (1 - koef)

In [ ]:
def make_edge_conv(in_ch, out_ch, kernel_size):
    """Make conv layer for image preprocessing 
    from 3 RGB channels to 9 spectral channels

    Args:
        in_ch (int): input image channels 
        out_ch (int): output channels (default = 9)
        kernel_size (int): conv kernel size

    Returns:
        edges (torch.nn.Conv2d): conv layer
    """

    pad = int((kernel_size - 1)/2)
    out_ch = 9
    edges = tr.nn.Conv2d(in_ch, out_ch, kernel_size, padding=pad).to(dev)
    types_params = t([[0, 0, pad, pad, 1]]).float()
    edge = create_conv_kernels(types_params, edges.weight.shape)  #.shape
    edges.weight.data = tr.zeros_like(edges.weight.data)

    for in_ch in range(edges.weight.data.shape[1]):
        edges.weight.data[in_ch*3+0, in_ch] = edge[0]
        edges.weight.data[in_ch*3+1, in_ch] = -edge[0]
        edges.weight.data[in_ch*3+2, in_ch] = edge[0] - edge[0].min()

    edges.weight.data /= edges.weight.data.max()
    return edges

In [ ]:
def show_imgs(X, y, pred, err, title):
    """Show random X image with true and pred mask

    Args:
        X (tensor)[batch_size, channels, H, W]: images
        y (tensor): true mask [batch_size, 1, H, W]
        pred (tensor): pred mask [batch_size, 1, H, W]
        title (str): image title to show
    """

    X = X.cpu().detach() 
    y = y.cpu().detach() 
    err = err.cpu().detach() 
    pred = (pred / pred.max() * 255).int().cpu().detach()
    i = tr.randint(X.shape[0], (1,)).item()
  
    plt.subplot(151)
    plt.title(title)
    plt.imshow(X[i, :3].permute(1,2,0))

    plt.subplot(152)
    plt.imshow(X[i, 3:6].permute(1,2,0))

    if 'torch' in err.type():
        plt.subplot(153)
        plt.imshow(err[i, 0])
    else:
        plt.subplot(153)
        plt.imshow(X[i, 6:9].permute(1,2,0))

    plt.subplot(154)
    plt.imshow(pred[i, 0])

    plt.subplot(155)
    plt.imshow(y[i, 0])

    plt.gcf().set_size_inches(20, 4)
    plt.show()

In [ ]:
def crit(y, pred, y_edges, edge_loss_koef, error = 0):
    """Loss func for images with importance koef of mask edges

     Args:
        y (tensor): true mask [batch_size, 1, H, W]
        pred (tensor): pred mask [batch_size, 1, H, W]
        y_edges (tensor): mask edges [batch_size, 1, H, W]
        edge_loss_koef (int): mask edges importance koef 

    Returns:
        loss (tensor): [1]
    """
    
    # (err + y - pred).abs().sum()
    edge_zone = (y_edges*(1 - edge_loss_koef) + edge_loss_koef)
    loss = ((edge_zone * (y - pred)).abs().sum() + (error - y + pred).abs().sum()) / tr.tensor(y.shape).prod() / 2
    return loss

In [ ]:
edges_x1 = make_edge_conv(3, 9, 5)
edges_x2 = make_edge_conv(3, 9, 9)
edges_x3 = make_edge_conv(3, 9, 21)

edges = tr.nn.Conv2d(1, 1, 21, padding=10).to(dev)
types_params = t([[0, 0, 10, 10, 1]]).float()
edges.weight.data[0] = create_conv_kernels(types_params, edges.weight.shape)

d = {k:v for v, k in enumerate(['iter', 'loss_valid', 'loss_valid_best', 
                        'loss', 'loss_train_best', 'dice', 
                        'time', 'time_iter', 'memory'])}

In [ ]:
train_loader = tr.utils.data.DataLoader(res[0], batch_size = 20, shuffle = True)
valid_loader = tr.utils.data.DataLoader(res[1], 20)

In [ ]:
models = {}
n_iters = 100
history = tr.zeros(1, n_iters, len(d))
timer_prev = time.time()
with_err = True
err = 0
loss_valid = 0
for model_num, p in enumerate(range(3, 7)):

    unet_params = [2**n for n in range(p, p + 5)]
    
    tr.cuda.empty_cache()
    model = Unet([1] + unet_params, 1, with_err).to(dev)
    # model_num = 2
    name = str(model.__class__()).split('(')[0]
    models[name] = {model_num:{'params':unet_params}}
    print('\n', name, unet_params)

    optim = tr.optim.Adam(model.parameters(), 0.0001)

    loss_train_best = 1e+10
    loss_valid_best = 1e+10

    blend_koef = 0
    edge_loss_koef = 0.7
    dropout_koef = 0.1
    aug_koef = 0.2
    history_model = tr.zeros(1, len(d), dtype=tr.float64)
    model_start = time.time()


    for iter in range(1, n_iters):
        model.train()
        for X, y in train_loader:
            X, y = X.to(dev), y.to(dev)
            # X = blend_xy(X, y, blend_koef)
            
            X, y = augments(X, y, aug_koef)  

            # if background.shape != X.shape:
            #     types_params = tr.randint(1, X.shape[-1], (t(X.shape[:2]).prod(), 5)).float()
            #     background = create_conv_kernels(types_params, X.shape).reshape(X.shape).to(dev)
            #     background /= background.max()
            # X[X == 0] = background[X == 0]

            X = tr.cat([edges_x1(X), edges_x2(X), edges_x3(X)], dim=-3)

            optim.zero_grad()
            if with_err:
                pred, err = model(X, dropout_koef)
            else:
                pred = model(X, dropout_koef)

            y_edges = edges(y).abs()
            loss = crit(y, pred, y_edges, edge_loss_koef, err)

            edge_loss_koef = 1 - loss.detach()
            aug_koef = loss.detach()
            # dropout_koef = loss.detach() / 100
            blend_koef = 1 - loss.detach() / 10

            loss.backward()
            optim.step()

            if loss < loss_train_best:
                model_train_best = model
                loss_train_best = loss   
                dice_train = dice(y, pred).item()  
                show_imgs(X, y,pred, err, f'Train at iter {iter}, train_loss {loss}, dice_train {dice_train}')
            
        model.eval()
        for X, y in valid_loader:
            X, y = X.to(dev), y.to(dev)
            X = tr.cat([edges_x1(X), edges_x2(X), edges_x3(X)], dim=-3)
            if with_err:
                pred, err = model(X, dropout_koef)
            else:
                pred = model(X, dropout_koef)

            y_edges = edges(y).abs()

            loss_valid = crit(y, pred, y_edges, edge_loss_koef, err)
            dice_valid = dice(y, pred).item()
            
            timer = time.time()
            time_iter = timer - timer_prev
            memory = round(tr.cuda.memory_allocated() * 1e-6)

            if loss_valid < loss_valid_best:
                model_valid_best = model
                loss_valid_best = loss_valid.item()
                models[name][model_num].update({
                    'loss_valid_best':loss_valid_best,
                    'dice_valid':dice_valid, 
                    'iter': iter, 
                    'time': timer - model_start, 
                    'model_valid_best':model_valid_best, })
                
                show_imgs(X, y, pred, err,  f'Valid at iter {iter}, val_loss {loss_valid}, dice_valid {dice_valid}')

            h = tr.tensor([iter, loss_valid, loss_valid_best, 
                            loss.item(), loss_train_best, dice_valid, 
                            timer, time_iter, memory])
            history_model = tr.cat([history_model, h.unsqueeze(0)])
            print(f"iter {h[d['iter']].int()} at {h[d['time_iter']].int()} sec, loss_valid {round(h[d['loss_valid']].item(), 2)}, dice {round(h[d['dice']].item(), 2)}, memory {h[d['memory']]} MB, loss {history_model[-1, d['loss']].item()}")
            timer_prev = timer
            break
    with open(path0 + '/data/models.pkl', 'wb') as fp:
        pickle.dump(models, fp)
    history = tr.cat([history, history_model.unsqueeze(0)])
    pred = model_valid_best(X, 0)
    show_imgs(X, y, pred, f'Valid at iter {iter}, val_loss {loss_valid}, dice {dice_valid}')


In [ ]:
with open(path0 + '/data/models.pkl', 'rb') as fp:
    models = pickle.load(fp)

In [ ]:
models['Unet'][0].keys()
best_model = models['Unet'][0]['model_valid_best']

In [ ]:
valid_loader = tr.utils.data.DataLoader(res[1], 10)
best_model.eval()
for X, y in valid_loader:
    X, y = X.to(dev), y.to(dev)
    X = tr.cat([edges_x1(X), edges_x2(X), edges_x3(X)], dim=-3)
    pred = best_model(X, 0)
    dice_valid = dice(y, pred).item()
    memory = round(tr.cuda.memory_allocated() * 1e-6)
    show_imgs(X, y, pred, f'Valid with dice {dice_valid}')

Сохрание результатов

In [ ]:
def id2rle(id):
    to_tensor_ = tvt.ToTensor()
    im = to_tensor_(Image.open(f"{path0}/data/valid/{id}.jpg")).unsqueeze(0).to(dev)
    X = tr.cat([edges_x1(im), edges_x2(im), edges_x3(im)], dim=-3)
    pred = (best_model(X, 0) > 0.5) * 1
    r = encode_rle(pred.cpu().detach())
    return r

Записываем csv c rle масками

In [ ]:
pred_pd['rle_mask'] = pred_pd.id.apply(id2rle)
pred_pd.to_csv(path0 + "/data/pred_valid.csv")

In [ ]:
def test2mask(path):
    to_tensor_ = tvt.ToTensor()
    im = to_tensor_(Image.open(path)).unsqueeze(0).to(dev)
    X = tr.cat([edges_x1(im), edges_x2(im), edges_x3(im)], dim=-3)
    pred = (best_model(X, 0) > 0.5).int() * 255
    return pred.squeeze().cpu().detach().numpy()

Сохраняем html c результатом на тестовой выборке

In [ ]:
paths_to_imgs = sorted(glob(path0 + "/data/test/*"))
pred_masks = [test2mask(path) for path in paths_to_imgs]
_ = get_html(paths_to_imgs, pred_masks, path_to_save = path0 + "/results/results_test")

Что можно добавить для дальнейшего повышения качества:

1. CRF которых неплохо делает постобработку.
2. модель с заменой фона и более креативной аугментацией.
3. ансамбли 
4. компрессию информации в маленькую якорную модель.
5. более сложные модели на основе преобразованний Фурье с подбором спектра и полноразмерными (как фото)  управляемыми сверточными ядрами Гаусса.
6. на Pyramid-подобные архитектуры
7. на графовые модели с инцедентными матрицами высокоуровневых признаков.
8. на трансформеры. 